In [ ]:
import numpy as np
from tqdm.auto import tqdm

In [ ]:
def is_valid_index(array, index):
    return (index >= 0).all() and (index < array.shape).all()

In [ ]:
def set_step(array_ref, array, index, value):
    if is_valid_index(array, index):
        if array_ref[*index] == value:
            array[*index] = value

## Part 1

In [ ]:
with open("data/day10/input.txt", "r") as file:
    map_raw = file.read()

In [ ]:
map_top = np.array([[int(y) for y in x] for x in map_raw.split("\n")])

In [ ]:
map_routes = []
for trailhead_ind in tqdm(np.argwhere(map_top == 0)):
    map_route = np.zeros_like(map_top, dtype=float)
    map_route[:] = np.nan
    map_route[*trailhead_ind] = 0

    for alt in range(10):
        alt_inds = np.argwhere(map_route == alt)
        for alt_ind in alt_inds:
            for offset in np.array([[0, 1], [0, -1], [1, 0], [-1, 0]]):
                set_step(map_top, map_route, alt_ind + offset, alt + 1)
    map_routes.append(map_route)

In [ ]:
sum([len(np.argwhere(map_route == 9)) for map_route in map_routes])

## Part 2

In [ ]:
def rating(map):
    map_dist = np.zeros_like(map, dtype=float)
    for alt in range(10):
        alt_inds = np.argwhere(map == alt)
        if alt == 0:
            map_dist[*alt_inds[0]] = 1
        else:
            map_dist_masked = np.ma.masked_where(map != alt - 1, map_dist).filled(0)
            for alt_ind in alt_inds:
                near_inds = [
                    alt_ind + offset
                    for offset in [[0, 1], [0, -1], [1, 0], [-1, 0]]
                    if is_valid_index(map, alt_ind + offset)
                ]
                map_dist[*alt_ind] = sum(
                    [map_dist_masked[*near_ind] for near_ind in near_inds]
                )

    return int(np.ma.masked_where(map != 9, map_dist).filled(0).sum())

In [ ]:
sum([rating(map_route) for map_route in tqdm(map_routes)])